<a href="https://colab.research.google.com/github/edermartelinho/Modelos_Linguagem_Neurais-LLM-/blob/main/Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#* Bibliotecas

In [ ]:
!pip install -U sentence-transformers
!pip install -U mteb
!pip install pysentimiento
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from huggingface_hub import hf_hub_download
import numpy as np
from sentence_transformers import SentenceTransformer,util
from transformers import AutoModel, AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import plotly.express as px
from pysentimiento import create_analyzer

#* Similaridade de cosseno

In [ ]:
#Exemplos de incorporação de palavras

word_embeddings = {
    "king": np.array([0.1, 0.2, 0.3]),
    "queen": np.array([0.15, 0.25, 0.35]),
    "man": np.array([0.4, 0.5, 0.6]),
    "woman": np.array([0.45, 0.55, 0.65]),
    "apple": np.array([0.7, 0.8, 0.9]),
    "orange": np.array([0.75, 0.85, 0.95]),
    "magesty":np.array([101,687,5923])
}

def cosine_sim(word1, word2):
  return cosine_similarity([word_embeddings["king"]], [word_embeddings["queen"]])[0][0]

similarity_King_queen = cosine_sim("King", "queen")
print(f"Similaridade de cosseno entre'King' e 'queen': {similarity_King_queen}")


##* A similaridade de coseno indica que os embedding 'King' e 'Queen' tem um grau de similaridade de 0.9974149030430577, enquanto 'man' e 'Woman' tem uma similaridade de 0.9998949782076886 e 'apple' e 'orange' apresenta uma similaridade de 0.9999823267952279, esta analise mostra que as palavras estão muito proxima em um espaco vetorial

In [ ]:
model = KeyedVectors.load_word2vec_format(hf_hub_download(repo_id="Word2vec/wikipedia2vec_ptwiki_20180420_100d", filename="ptwiki_20180420_100d.txt"))
#model.most_similar("sua_palavra")

#Supondo que você baixou o modelo corretamente e o carregou em uma variável chamada 'modelo'

similar_words = model.most_similar('carro', topn=5)  # Obtenha as 5 principais palavras semelhantes
print(f"Words similar to 'carro': {similar_words}")

result = model.most_similar(positive=['carro', 'fusca'], negative=['trator'], topn=1)
print(f"carro - trator + fusca = {result[0][0]}")  # Imprima a palavra mais semelhante



#*  word2vec
É uma tecnica de aprendizado de unidades de representações distribuidas proposta por Mikolov et al(2013)que tem como objetivo capturar a semântica e a relação entre unidades de representação em um corpus, aprendendo embeddings estaticos para cada palavra presente no vocabulario de treino(Jurasfsky;Martin,2023) a ideia é que palavras que ocorrem em contexto semelhantes tenham significados semelhantes.



#* Gensim
O gensim é uma biblioteca python para modelagem de topicos, indexação de documentos e recuperação de similaridade com grandes corporas.

Ao usar o modelo: from gensim.models import keyedVectors percebi que a maior similaridade no top 5 foi 'caminhão' com similaridade de 0.8685649633407593, percebo tambem que quando coloco trator como negativo o modelo me sugere um carro mais moderno

In [ ]:
model = SentenceTransformer('mteb-pt/average_pt_nilc_glove_s1000')
# Função para calcular similaridade de cosseno
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# a analogia: “mar” está para “baleia” assim como “passaro” está para ?
#mar = modelo['mar']
#baleia = modelo['baleia']
#passaro = modelo['passaro']

mar = model.encode(['mar'])[0] # Use .encode() e obtenha o primeiro elemento
baleia = model.encode(['baleia'])[0] # Use .encode() e obtennha o segundo elemento
passaro = model.encode(['passaro'])[0] # Use .encode() e obtenha o primei elemento



result_vector = passaro - mar + baleia

vocabulary = ['aguia','pombo','tubarão','gaivota']


max_similarity = -1
most_similar_word = ''
for word in vocabulary:

       similarity = util.cos_sim(result_vector, model.encode([word])[0]).item()
       if similarity > max_similarity:
            max_similarity = similarity
            most_similar_word = word

print(f"'mar' is to 'baleia' as 'passaro' is to '{most_similar_word}'")

# * GLOve
##O GLOve adota uma perspectiva global, levando em consideração a contagem de coocorência palavra-palavra em um corpus. Essa abordagem permite que o glove capture informações de relação semântica e sintática entre as palavras.





In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-en-pt-cased")
model = AutoModel.from_pretrained("Geotrend/bert-base-en-pt-cased")


print(f"Using model: {tokenizer}")


##*Importei um tokenizador, define a lista de palavras para explorar e tokenizei,foi extraido o ultimo estado oculto e calculada a media para obter uma unica incorporação para cada palavr da lista, calculei a similaridade de coseno, aplyquei o pca e plotei os imbedin, define algumas sentenças, tokenizeie gerei embeddings, extrai a media do ultimo estado oculto e plotei embeddings de frasese em 2d usando PCA, tambem usei as mesmas sentenças para fazer analise de sentimento mostrando o resultado executando a celula.

In [ ]:
words = ["mar", "maré", "maremoto", "marrevolto", "carro","caminhão","caminhoneiro", "fusca", "carroça", "fiat147"]

# Tokenizar e gerar embeddings
inputs = tokenizer(words, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)

# Extraia o último estado oculto e calcule a média para obter uma única incorporação para cada palavra
embeddings = outputs.last_hidden_state.mean(dim=1)
print("Embeddings shape:", embeddings.shape)

In [ ]:
similarity_matrix = cosine_similarity(embeddings)
print("Cosine Similarity Matrix:\n", similarity_matrix)


#*A Análise de Componentes Principais (ACP) ou Principal Component Analysis (PCA)

##É um procedimento matemático que utiliza uma transformação ortogonal (ortogonalização de vetores) para converter um conjunto de observações de variáveis possivelmente correlacionadas num conjunto de valores de variáveis linearmente não correlacionadas chamadas de componentes principais

In [ ]:
# Aplicando PCA
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Traçando os embeddings
plt.figure(figsize=(8, 6))
for i, word in enumerate(words):
    x, y = embeddings_2d[i]
    plt.scatter(x, y)
    plt.text(x + 0.02, y + 0.02, word, fontsize=12)

plt.title("2D Visualization of Word Embeddings")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.grid()
plt.show()


#*Incorporação de vizinho estocástico distribuída em T.

#t-SNE  é uma ferramenta para visualizar dados de alta dimensão. Ele converte semelhanças entre pontos de dados em probabilidades conjuntas e tenta minimizar a divergência de Kullback-Leibler entre as probabilidades conjuntas da incorporação de

In [ ]:
 tsne = TSNE(n_components=3, perplexity= 5 )
 projections = tsne.fit_transform(embeddings)

 fig = px.scatter_3d(projections, x=0, y=1, z = 2, color=words)
 fig.show()

In [ ]:
# Defina frases com semelhança semântica
sentences = ["As eleiçoes foram acirradas entre os partidos.",
             "Alguns partidos reelegeram seus prefeitos.",
             "A abstenção nas eleições gerou debates",
             "O derretimento das calotas polares é preocupante.",
             "O degelo muda o clima.",
             "Efeitos do aquecimento global."
             ]

# Tokenizar e gerar embeddings
sentence_inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    sentence_outputs = model(**sentence_inputs)

#Extrai a média do último estado oculto
sentence_embeddings = sentence_outputs.last_hidden_state.mean(dim=1)

# Visualize incorporações de frases em 2D usando PCA
sentence_embeddings_2d = pca.fit_transform(sentence_embeddings)

plt.figure(figsize=(8, 6))
for i, sentence in enumerate(sentences):
    x, y = sentence_embeddings_2d[i]
    plt.scatter(x, y)
    plt.text(x + 0.02, y + 0.02, sentence, fontsize=10)

plt.title("2D Visualization of Sentence Embeddings")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.grid()
plt.show()



In [ ]:
 tsne = TSNE(n_components=3, perplexity= 5 )
 projections = tsne.fit_transform(sentence_embeddings)

 num_points = projections.shape[0]  # Get the number of data points
 colors =sentences[:num_points]  # Slice the words list to match the numbe

 fig = px.scatter_3d(projections, x=0, y=1, z = 2, color=colors)
 fig.show()

#*Analise de Sentimento

#A análise de sentimentos tem várias aplicações práticas, desde o monitoramento de marcas até a análise de comentários de clientes.

In [ ]:
analyzer = create_analyzer(task="sentiment", lang="pt")
# Chame o método 'predict' entre parênteses e passe a lista de sentenças
for sentence in sentences:
    sentiment = analyzer.predict(sentence)
    print(f"Sentence: {sentence}")
    print(f"Sentiment: {sentiment.output}")  # Access the sentiment output

# retorna AnalyzerOutput(saída=POS, probas={POS: 0,998, NEG: 0,002, NEU: 0,000})
